# Ex5: Weighting
This example demonstrates how weighting can be used to balance the fit accuracy between large and small magnitudes of the response.

Because the vector fitting algorithm uses a least squares fit, large values in the responses contribute more to the least squares error than small values, thus giving them by default more weight.

But sometimes we are interested to accurately fit very small magnitudes of a response very accurately, for example when analyzing the cross-talk or isolation between certain ports of a network.

In this case, we can weight every sample by multiplying it with its inverse magnitude in the least squares fit and so we will get the same relative accuracy for large and small values. 

In [ ]:
import matplotlib.pyplot as mplt
import numpy as np

import skrf

nw = skrf.network.Network('./190ghz_tx_measured.S2P')
vf = skrf.VectorFitting(nw)
vf.auto_fit(verbose=True)

We can then plot the fitted responsse

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

We can already see that the response i=0, j=1 shows a deviation to the data of around 10 dB at 1.6 GHz and the deep notch in the data that goes down to -90 dB is actually only a notch down to -65 dB, so we miss the notch by 25 dB!

We can also see that the relative error for response 0, 1 is the heighest.

Now we run a new fit with inverse_magnited weighting to improve this.

In [ ]:
vf.auto_fit(weighting='inverse_magnitude', verbose=True)

We can already see that the total relative error decreased a lot compared to the non-weigthed fit and also the relative error for response 0, 1 is now about half.
If we now plot the responses, we can see the difference:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

We can observe from the plot that the strong deviations in response 0, 1 around 1.6 GHz is now gone completely and the -90 dB notch is exactly represented by the model.

We can also observe in the response 1,1 that the fit deviates a bit more than in the non-weigthed fit from the data around 1.4 GHz. This is normal and comes because we gave more weight to the small values. The least squares fit will sacrifice a bit of the accuracy in the large magnitudes in favor of the accuracy in the small magnitudes. 

This effect of decreased accuracy in large magnitudes in favor of better accuracy for lower magnitudes is actually a trade off, that can be adjusted with a parameter called weighting_accuracy_db. This parameter limits the magnitude weighting to a certain level, for example -40 dB. Every magnitude below this level is not weighted by more than the limit. 

This is very useful because, for example, we might not actually be interested in fitting the -90 dB notch exactly, but still we might be interested in fitting very exact down to about -40 dB. If we specify this as a parameter, we will get good accuracy down to -40 dB without sacrificing more than neccesary of the accuracy for the large values!

Let's try this and see what we can get:

In [ ]:
vf.auto_fit(weighting='inverse_magnitude', weighting_accuracy_db=-40, verbose=True)

Now the relative error of the response 0, 1 went up again, which is expected, but let's see how it looks in the plots:

In [ ]:
freqs = np.linspace(np.min(vf.network.f), np.max(vf.network.f), 201)
n_ports=vf._get_n_ports()
fig, ax = mplt.subplots(n_ports, n_ports)
fig.set_size_inches(12, 8)
for i in range(n_ports):
    for j in range(n_ports):
        vf.plot_s_db(i, j, freqs=freqs, ax=ax[i][j])
fig.tight_layout()
mplt.show()

We can now see that we could maintain the good fit around 1.6 GHz in the -55 dB range, but we lost (as expected) the perfect match of the -90 dB notch. But the good thing is that we also got back our good fit for the response 1, 1!

Thus, the specification of the weighting_accuracy_db parameter is very important and it should not be set much lower than necessary to achieve the desired accuracy down to a certain magnitude.

Setting weighted_accuracy_db=0 is thus equivalent to non-weighted vector fitting.

It is also possible to set weighting='uniform', which will weight all samples with 1, thus, not weighting them at all. This is also equivalent to the non-weighted vector fitting. In the future, other weighting schemes can easily be implemented if desired.

Moreover, the weights can directly be providid as an argument and thus, arbitrary user defined weights can be used. Details on this can be found in the documentation of the arguments of auto_fit().